In [1]:
# ==============================================================================
# 핀테크 지원센터 뉴스 데이터 수집 파이프라인
# (18 페이지 반복 -> 주요뉴스 게시글 진입 -> 개별 원문 링크 추출 -> 본문 텍스트 수집)
# ==============================================================================

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
# from tqdm import tqdm # tqdm 라이브러리 사용 제거

# 기본 URL 설정
LIST_URL = "https://fintech.or.kr/web/board/boardContentsList.do"  # 게시글 목록 URL
VIEW_URL = "https://fintech.or.kr/web/board/boardContentsView.do"  # 게시글 상세 보기 URL

# 최종 수집 결과를 저장할 리스트
final_data = []

# ==============================================================================
# 1단계: 전체 18 페이지 순회하며 '핀테크 주요뉴스' 게시글 링크 수집
# ==============================================================================
def scrape_major_news_list(start_page, end_page):
    """
    1페이지부터 18페이지까지 순회하며 '핀테크 주요뉴스' 게시글의 제목, 날짜, 내부 링크(JavaScript)를 수집합니다.
    """
    print(f"1. 주요뉴스 게시글 목록 수집 시작 (페이지 {start_page} ~ {end_page})")
    
    major_news_posts = []
    
    # 1. 18 페이지 반복문 시작
    for page in range(start_page, end_page + 1):
        try:
            # 페이지 파라미터
            params = { "board_id": "6", "menu_id": "6500", "miv_pageNo": page }
            r = requests.get(LIST_URL, params=params, timeout=5)
            r.raise_for_status()
            soup = bs(r.content, "lxml")
            
            # 게시글 제목 a 태그 선택
            page_list = soup.select(".title a")
            
            for a_tag in page_list:
                title = a_tag.get_text(strip=True)
                
                # '핀테크 주요뉴스' 제목 필터링
                if "핀테크 주요뉴스" in title:
                    link_js = a_tag.get("href")  # ex: javascript:fn_view('contents_id', '6')
                    
                    # 날짜 추출: 동일 tr 내의 마지막 td
                    tr_tag = a_tag.find_parent("tr")
                    date = tr_tag.select("td")[-1].get_text(strip=True) if tr_tag else ""
                    
                    # contents_id 추출 (개별 뉴스 원문 목록을 가져오기 위해 필요)
                    match = re.search(r"fn_view\('([a-f0-9]+)'", link_js)
                    contents_id = match.group(1) if match else None
                    
                    if contents_id:
                        major_news_posts.append({
                            'major_title': title,
                            'major_date': date,
                            'contents_id': contents_id
                        })
            
            time.sleep(0.5) # 서버 부하 방지를 위한 딜레이
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            continue
            
    print(f"-> 총 {len(major_news_posts)}개의 주요뉴스 게시글 목록 수집 완료.")
    return major_news_posts

# ==============================================================================
# 2단계: 주요뉴스 게시글 진입 및 개별 뉴스 원문 링크 추출
# ==============================================================================
def scrape_individual_news_links(contents_id, major_title, major_date):
    """
    주요뉴스 게시글(contents_id)에 POST 요청을 보내 개별 뉴스 원문 링크와 제목을 추출합니다.
    """
    post_payload = {
        "method": "searchCorpList", 
        "orderMode": 3, 
        "orderStat": "D",
        "searchType": 13, 
        "fiscalYearEnd": "all", 
        "location": "all", 
        "contents_id": contents_id,  # 동적으로 설정
        "homepage": "F", 
        "up_menu_if": 6000
    }
    
    try:
        r = requests.post(VIEW_URL, data=post_payload, timeout=10)
        r.raise_for_status()
        soup = bs(r.content, 'lxml')
        
        individual_news = []
        # 원문 링크는 게시글 내용 영역의 <a> 태그에 있습니다.
        fintech_list = soup.select("td a")
        
        for a_tag in fintech_list:
            news_title = a_tag.text.strip()
            news_link = a_tag.get('href', '').strip()
            
            # 유효한 HTTP 링크만 수집하고, 제목이 비어있지 않으며, 불필요한 Footer 링크 제거
            if news_link.startswith('http') and news_title and news_title != '이전':
                individual_news.append({
                    'date': major_date,
                    'major_title': major_title,
                    'news_title': news_title,
                    'news_link': news_link
                })
        
        return individual_news

    except requests.exceptions.RequestException as e:
        print(f"Error fetching contents_id {contents_id}: {e}")
        return []

# ==============================================================================
# 3단계: 외부 뉴스 원문 링크에서 본문 텍스트 추출 (사용자 제공 함수 기반)
# ==============================================================================
def get_article_text(url):
    """
    외부 뉴스 원문 링크에서 본문 텍스트 추출 (타임아웃 및 예외 처리 포함)
    """
    try:
        # 대부분의 뉴스 사이트는 User-Agent가 없으면 접근이 막힙니다.
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        r = requests.get(url, timeout=10, headers=headers)
        r.encoding = 'utf-8' # 인코딩 설정
        r.raise_for_status()
        soup = bs(r.text, 'lxml')

        # 네이버, 다음 등 일반적인 뉴스 기사 본문 추출을 위한 선택자 시도
        # dic_area는 네이버 뉴스 본문 선택자
        paragraphs = soup.select('#dic_area, .article_view, #articeBody, #articleBodyContents')
        
        if paragraphs:
            # 추출된 모든 단락을 줄바꿈으로 연결
            content = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
            return content
        else:
            # 본문 선택자로 찾지 못했을 경우, 문서 전체 텍스트 반환 (잡음 많음)
            return soup.body.get_text(strip=True)[:1000] # 너무 길어지는 것을 방지

    except Exception as e:
        # print(f"Error fetching {url}: {e}") # 디버깅 시에만 사용
        return ""


# ==============================================================================
# 4단계: 전체 파이프라인 실행
# 이 부분이 주요뉴스 게시글(contents_id)을 반복하고, 
# 그 안에서 개별 뉴스 원문 링크(news_link)를 반복하여 본문을 추출하는 핵심 로직입니다.
# ==============================================================================

# 1. 주요 뉴스 게시글 목록 수집 (1페이지부터 18페이지까지)
major_posts = scrape_major_news_list(start_page=1, end_page=18)

# 2. 각 게시글에 진입하여 개별 뉴스 링크 수집 및 본문 추출
print("\n2. 개별 뉴스 원문 링크 추출 및 본문 수집 시작 (총 뉴스 게시글 수:", len(major_posts), ")")

# 진행 상황을 표시하기 위해 수동 카운터 사용
for i, post in enumerate(major_posts): # <-- 2. '핀테크 주요뉴스' 게시글 반복
    
    # 현재 진행 상황 출력
    print(f"[{i+1}/{len(major_posts)}] 게시글 수집 중: {post['major_title']} ({post['major_date']})")
    
    # 2단계 실행: 개별 뉴스 원문 링크 추출
    individual_news_list = scrape_individual_news_links(
        contents_id=post['contents_id'], 
        major_title=post['major_title'], 
        major_date=post['major_date']
    )
    
    # 3단계 실행: 각 원문 링크에서 본문 텍스트 추출
    for news_item in individual_news_list: # <-- 3. 개별 뉴스 원문 링크 반복
        article_body = get_article_text(news_item['news_link'])
        
        # 본문이 성공적으로 추출된 경우에만 최종 데이터에 추가
        if article_body:
            final_data.append({
                'date': news_item['date'],
                'major_title': news_item['major_title'],
                'title': news_item['news_title'],
                'link': news_item['news_link'],
                'body': article_body
            })
    
    time.sleep(1) # 각 주요뉴스 게시글 사이의 딜레이

# ==============================================================================
# 5단계: 결과 정리 및 저장
# ==============================================================================

df_final = pd.DataFrame(final_data)

print("\n==============================================================================")
print(f"🔥 최종 수집 완료! 총 {len(df_final)}개의 뉴스 기사 본문 수집됨.")
print("==============================================================================")
print("\n[수집된 데이터프레임 미리보기]")
print(df_final.head())
print("\n[데이터프레임 정보]")
print(df_final.info())

# 결과를 CSV 파일로 저장
CSV_FILE = 'fintech_data_raw.csv'
df_final.to_csv(CSV_FILE, index=False, encoding='utf-8-sig')
print(f"\n데이터가 '{CSV_FILE}' 파일로 저장되었습니다. 이제 이 데이터를 바탕으로 분석을 진행할 수 있습니다.")

1. 주요뉴스 게시글 목록 수집 시작 (페이지 1 ~ 18)
-> 총 0개의 주요뉴스 게시글 목록 수집 완료.

2. 개별 뉴스 원문 링크 추출 및 본문 수집 시작 (총 뉴스 게시글 수: 0 )

🔥 최종 수집 완료! 총 0개의 뉴스 기사 본문 수집됨.

[수집된 데이터프레임 미리보기]
Empty DataFrame
Columns: []
Index: []

[데이터프레임 정보]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None

데이터가 'fintech_data_raw.csv' 파일로 저장되었습니다. 이제 이 데이터를 바탕으로 분석을 진행할 수 있습니다.
